# Load Token Dictionnaire

In [2]:
DICT_PATH = "tokenizer_dict.pkl"
MODEL_PATH = 'knn_model.joblib'
NUM_PREPROCSS_PATH = "numeric_transformer.joblib"
CAT_PREPROCSS_PATH = "categorical_transformer.joblib"

BASE_PATH = "modeldata/"

import pickle
import joblib

dictionary={}
with open(BASE_PATH+DICT_PATH, 'rb') as f:
    dictionary=pickle.load(f)
knn = joblib.load(BASE_PATH+"knn_model.joblib")

numeric_transformer=joblib.load(BASE_PATH+NUM_PREPROCSS_PATH)
categorical_transformer=joblib.load(BASE_PATH+CAT_PREPROCSS_PATH)

dictionary


{'``': 1,
 'patient': 2,
 'souffr': 3,
 'maux': 4,
 'têt': 5,
 'fréquent': 6,
 'fatigu': 7,
 '.': 8,
 "''": 9,
 'a': 10,
 'glycem': 11,
 'élev': 12,
 'douleur': 13,
 'abdominal': 14,
 'musculair': 15,
 'apres': 16,
 "l'exercic": 17,
 'plaint': 18,
 'vertig': 19,
 'palpit': 20,
 'hypertens': 21,
 'mal': 22,
 'contrôl': 23,
 'hypercholestérolem': 24,
 'articulair': 25,
 'raideur': 26,
 'présent': 27,
 'symptôm': 28,
 'diabet': 29,
 'pelvien': 30,
 'cramp': 31,
 'thorac': 32,
 'dyspn': 33,
 "l'effort": 34,
 'dépress': 35,
 "d'anxiet": 36,
 'indigest': 37,
 'bouff': 38,
 'chaleur': 39,
 'sueur': 40,
 'nocturn': 41,
 'engourd': 42,
 'extrem': 43,
 'migrain': 44,
 'séver': 45,
 'toux': 46,
 'persist': 47,
 'lombair': 48,
 'chroniqu': 49,
 'neuropath': 50,
 'pied': 51,
 'sinusit': 52,
 'insuffis': 53,
 'cardiaqu': 54,
 'congest': 55,
 'cervical': 56,
 'allerg': 57,
 'saisonni': 58,
 'menstruel': 59,
 'troubl': 60,
 'sommeil': 61,
 "l'anxiet": 62,
 'hyperglycem': 63,
 'dyslipidem': 64,
 'saign

In [5]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
# load data
import numpy as np
import pandas as pd

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')

# Initialize NLTK's SnowballStemmer and stopwords for French
stemmer = SnowballStemmer('french')
stop_words = set(stopwords.words('french'))

def tokenizeText(text, dictionary):
    # Convert text to lowercase
    text = text.lower()
    
    # Tokenize text using NLTK tokenizer
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    
    # Apply stemming using SnowballStemmer
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    
    # Numerize tokens
    numerical_tokens = []
    for token in stemmed_tokens:
        if token not in dictionary:
            # dictionary[token] = torch.tensor(len(dictionary) + 1,dtype=torch.int32)
            dictionary[token] = len(dictionary) + 1
        numerical_tokens.append(dictionary[token])
    
    # Convert numerical tokens and dictionary values to tensors
    # numerical_tokens_tensor = torch.tensor(numerical_tokens)
    numerical_tokens_np = np.array(numerical_tokens)
    # return numerical_tokens_tensor
    return numerical_tokens_np



[nltk_data] Downloading package punkt to C:\Users\Aziz
[nltk_data]     Hlila\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Aziz
[nltk_data]     Hlila\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def pad_sequences(sequence_list, maxlen=None):
    if not maxlen:
        maxlen = max(len(seq) for seq in sequence_list)
    padded_sequences = np.zeros((len(sequence_list), maxlen), dtype=int)
    for i, seq in enumerate(sequence_list):
        if len(seq) > 0:  # Check if the sequence is not empty
            padded_sequences[i, :len(seq)] = seq
    return padded_sequences




In [16]:
import pandas as pd

X = pd.read_csv("validation.csv")

In [17]:
descriptionsTokenized = X['Description'].apply(lambda x: tokenizeText(x, dictionary))
descriptionsTokenized

0    [1, 2, 3, 4, 5, 6, 7, 8, 9]
Name: Description, dtype: object

In [18]:
descriptionsTokenizedPadded = pad_sequences(descriptionsTokenized,300)


In [21]:
from sklearn.pipeline import Pipeline
import numpy as np 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

numeric_features = ['tailleCm', 'poidsKg', 'IMC', 'nbGrossesse', 'nbEnfantsVivants',
                    'nbMacrosomies', 'nbAvortements', 'nbMortNes', 'ageMenopause',
                    'alcoolSemaine', 'nbCigaretteParJour', 'Age']
categorical_features = ['groupeSanguin', 'HTA', 'diabete', 'dyslipidemie', 'tabacStatus',
                        'drogue']

numeric_transformed = numeric_transformer.transform(X[numeric_features].to_numpy())

categorical_transformed  = categorical_transformer.transform(X[categorical_features].to_numpy())

preprocessed_data = np.hstack((numeric_transformed, categorical_transformed.toarray()))
X_combined = np.hstack((preprocessed_data, descriptionsTokenizedPadded))


In [35]:

distances, indices = knn.kneighbors(X_combined, n_neighbors=10)
# Convert indices to actual predictions



array([[ 0.        , 12.32882801, 16.39980687, 16.39980687, 16.39980687,
        18.38849449, 20.41973872, 20.9001628 , 20.9001628 , 22.15747426]])